# 상권 / 업종 정보 확인을 위한 전처리

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

drive_path = '/content/drive/MyDrive/new_project1/'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np

import re
import os

import geopandas as gpd



# 제출용 코드에서 깔끔하게 보이게,..
import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 인허가 정보 파일 확인
file_list = os.listdir(f'{drive_path}data/local')
file_list[:5]

['6110000_서울특별시_03_10_01_P_비디오물감상실업.csv',
 '6110000_서울특별시_03_10_02_P_비디오물배급업.csv',
 '6110000_서울특별시_03_10_03_P_비디오물소극장업.csv',
 '6110000_서울특별시_03_10_04_P_비디오물시청제공업.csv',
 '6110000_서울특별시_03_10_05_P_비디오물제작업.csv']

## 모든 업종을 대상으로 하는 것이 아니라 필요한 업종 선정
- 편의점은 휴게음식점의 세부 업종에 포함되어 있다.

In [ ]:
# re.sub로 한글만 남기려 했는데 왠지 모르게 적용이 안되서
# 다른 방법을 이용

type_list = []

for csv in file_list:
    type_name = csv[len('6110000_서울특별시_03_10_01_P_'):-4]
    type_list.append(type_name)

type_list[:5]

['비디오물감상실업',
 '비디오물배급업',
 '비디오물소극장업',
 '비디오물시청제공업',
 '비디오물제작업']

In [ ]:
print(type_list)

['비디오물감상실업', '비디오물배급업', '비디오물소극장업', '비디오물시청제공업', '비디오물제작업', '관광숙박업', '관광펜션업', '외국인관광도시민박업', '숙박업', '국내여행업', '자동차야영장업', '한옥체험업', '종합여행업', '영화배급업', '영화수입업', '영화상영관', '영화상영업', '영화제작업', '온라인음악서비스제공업', '음반.음악영상물배급업', '음반물배급업', '음반.음악영상물제작업', '음반물제작업', '옥외광고업', '인쇄사', '출판사', '미용업', '이용업', '의료기관세탁물처리업', '세탁업', '위탁급식영업', '집단급식소식품판매업', '집단급식소', '건강기능식품유통전문판매업', '건강기능식품일반판매업', '식육포장처리업', '식품냉동냉장업', '축산판매업', '식품운반업', '식품소분업', '식품자동판매기업', '식품첨가물제조업', '식품판매업기타', '식품제조가공업', '옹기류제조업', '용기·포장지제조업', '유통저

In [ ]:
## 상권 분석에 필요한 업종 선정
store_type = ['영화상영관', '미용업', '이용업','세탁업', '일반음식점', '휴게음식점', '제과점영업', '동물병원', '동물미용업', '병원', '의원','약국']

In [ ]:
selected_csv = [csv for csv in file_list
                    if any(store_type_item in csv
                         for store_type_item in store_type)]
selected_csv

['6110000_서울특별시_03_13_02_P_영화상영관.csv',
 '6110000_서울특별시_05_18_01_P_미용업.csv',
 '6110000_서울특별시_05_19_01_P_이용업.csv',
 '6110000_서울특별시_06_20_01_P_세탁업.csv',
 '6110000_서울특별시_07_24_04_P_일반음식점.csv',
 '6110000_서울특별시_07_24_05_P_휴게음식점.csv',
 '6110000_서울특별시_07_22_18_P_제과점영업.csv',
 '6110000_서울특별시_02_03_11_P_동물미용업.csv',
 '6110000_서울특별시_01_01_01_P_병원.csv',
 '6110000_서울특별시_01_01_02_P_의원.csv',
 '6110000_서울특별시_01_01_06_P_약국.csv',
 '6110000_서울특별시_02_03_01_P_동물병원.csv',
 '6110000_서울특별시_02_03_02_P_동물약국.csv']

In [ ]:
## 위경도 변환을 위한 준비 - 나중에 시각화를 위해서
from pyproj import Transformer
coord_origin_type = "epsg:2097"
coord_trans_type =  "epsg:4326"

def coord_trans(coord, coord_orgin, coord_trans):
    y_, x_ = Transformer.from_crs(coord_origin_type, coord_trans_type).transform(coord[:,0], coord[:, 1])
    temp = np.dstack([y_, x_])[0]
    return pd.DataFrame(temp, columns=['lat', 'lon'])

In [ ]:
column_list = ['번호','개방서비스명','인허가일자', '인허가취소일자', '영업상태구분코드', '영업상태명', '상세영업상태코드', '폐업일자',
               '소재지전체주소', '도로명전체주소', '사업장명', '업태구분명', '좌표정보(x)', '좌표정보(y)', '업종']

df = pd.DataFrame(columns = column_list)

In [ ]:
for file_name in selected_csv:

    folder_path = f'{drive_path}data/local/'
    path = folder_path + file_name

    temp = pd.read_csv(path, encoding = 'cp949', low_memory = False)

    # 업태구분명이 -> 중분류
    # 업종 -> 소분류
    temp['업태구분명'] = temp.apply(lambda x: x['개방서비스명'] if pd.isna(x['업태구분명']) else x['업태구분명'], axis = 1)
    temp['업종'] = temp.apply(lambda x: x['개방서비스명'] + ' ' + x['업태구분명'], axis = 1)

    # 모든 데이터를 같은 양식으로 바꾸기 위한 준비
    temp = temp[column_list]

    ### 소재지 주소에서 시군구/읍면동 추출 - 모든 데이터는 서울시 데이터라 시도 구분은 필요 없음
    temp['시군구'] = temp['소재지전체주소'].apply(lambda x: x.split()[1]  if isinstance(x, str) and len(x.split()) > 1 else None)
    temp['읍면동']  = temp['소재지전체주소'].apply(lambda x: x.split()[2]  if isinstance(x, str) and len(x.split()) > 2 else None)

    ## 주소지가 명확하기 않은 경우
    # 주소 정보를 가지고 도로명주소 api를 활용해 주소 확보
    # 좌표 정보를 통해 소재지 추정
    ### 이번 프로젝트에서는 미실시 .... 해당 건수가 많지만 전체에서 5%미만


    # ##### 좌표 변환 과정 - data(epsg:2097) / 변환(epsg:4326)
    # temp[['좌표정보(x)', '좌표정보(y)']]에서 좌표 정보가 잘못입력되어 숫자가 아닌경우 존재
    temp['좌표정보(x)'] = temp['좌표정보(x)'].apply(lambda x: float(x) if not isinstance(x, str) else np.nan)
    temp['좌표정보(y)'] = temp['좌표정보(y)'].apply(lambda x: float(x) if not isinstance(x, str) else np.nan)

    ### 좌표변환
    temp_coord = temp[['좌표정보(x)', '좌표정보(y)']]
    temp_coord = np.array(temp_coord)

    # 좌표 변환
    # y_, x_ = Transformer.from_crs(coord_origin_type, coord_trans_type).transform(temp_coord[0,0], temp_coord[0,1])
    # 위 방법을 함수화(coord_trans) 해서 사용
    temp_coord = coord_trans(temp_coord, coord_origin_type, coord_trans_type)

    temp_merge = pd.concat([temp, temp_coord], axis = 1)


    ###  인허가 / 폐업일자 확인 -> 연도별 영업 상태 확인을 위해서
    #-> 다른 자료들이 시계열 자료가 아니므로 이번 프로젝트에서 미사용

    temp_merge['인허가일자'] = pd.to_datetime(temp_merge['인허가일자'], errors='coerce')

    # 폐업일자가 없는 경우 현재 영업중으로 간주
    # 폐업일자 nan -> 2024-01-01로
    temp_merge['폐업일자'] = temp_merge['폐업일자'].apply(lambda x: '2024-01-01' if pd.isna(x) else x)
    temp_merge['폐업일자'] = pd.to_datetime(temp_merge['폐업일자'], errors='coerce')

    # # 연도별 해당 업체 운영 여부 확인
    # for i in range(2016, 2024):
    #     temp_var1 = 'run' + str(i)
    #     open_date = str(i) + '-12-30'
    #     close_date = str(i) + '-12-31'
    #     temp_merge[temp_var1] = temp_merge.apply(lambda x: 1 if ((x['인허가일자'] <= pd.Timestamp(open_date)) &
    #                                                         (x['폐업일자'] > pd.Timestamp(close_date)))
    #                                                         else 0,
    #                                                     axis=1)

    df = pd.concat([df, temp_merge], axis = 0)

In [ ]:
df.columns

Index(['번호', '개방서비스명', '인허가일자', '인허가취소일자', '영업상태구분코드', '영업상태명', '상세영업상태코드',
       '폐업일자', '소재지전체주소', '도로명전체주소', '사업장명', '업태구분명', '좌표정보(x)', '좌표정보(y)',
       '업종', '시군구', '읍면동', 'lat', 'lon'],
      dtype='object')

In [ ]:
# 2023년에 운영하는 상업기간 확인
df = df[df['영업상태구분코드'] == 1]

In [ ]:
df.head()

,번호,개방서비스명,인허가일자,인허가취소일자,영업상태구분코드,영업상태명,상세영업상태코드,폐업일자,소재지전체주소,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),업종,시군구,읍면동,lat,lon
41,42,영화상영관,2003-02-12,NaN,1,영업/정상,13,2024-01-01,서울특별시 마포구 동교동 166-14 동교동 스타피카소,"서울특별시 마포구 양화로 176, 동교동 스타피카소 8층 (동교동)",독립영화전용관 인디스페이스,영화상영관,193282.130800,450549.280167,영화상영관 영화상영관,마포구,동교동,35.207554,129.749109
42,43,영화상영관,2003-02-12,NaN,1,영업/정상,13,2024-01-01,서울특별시 종로구 낙원동 284-6번지,서울특별시 종로구 삼일대로 428 (낙원동),허리우드클래식,영화상영관,198852.959717,452253.942585,영화상영관 영화상영관,종로구,낙원동,35.257281,129.769514
43,44,영화상영관,2003-02-12,NaN,1,영업/정상,13,2024-01-01,서울특별시 종로구 낙원동 284-6번지,"서울특별시 종로구 삼일대로 428, 4층 (낙원동)",낭만극장,영화상영관,198852.959717,452253.942585,영화상영관 영화상영관,종로구,낙원동,35.257281,129.769514
44,45,영화상영관,2003-02-12,NaN,1,영업/정상,13,2024-01-01,서울특별시 종로구 신문로1가 226 흥국생명빌딩,"서울특별시 종로구 새문안로 68, 흥국생명빌딩 지하2층 (신문로1가)",씨네큐브광화문 1관,영화상영관,197482.205330,451938.307925,영화상영관 영화상영관,종로구,신문로1가,35.245019,129.765630
45,46,영화상영관,2003-02-12,NaN,1,영업/정상,13,2024-01-01,서울특별시 종로구 신문로1가 226 흥국생명빌딩,"서울특별시 종로구 새문안로 68, 흥국생명빌딩 지하2층 (신문로1가)",씨네큐브광화문 2관,영화상영관,197482.205330,451938.307925,영화상영관 영화상영관,종로구,신문로1가,35.245019,129.765630


## 추출된 주소 정보를 동별 통계로 변환하기 위해
- 도로명 주소를 기반으로 변환

In [ ]:
dong_convert = pd.read_csv(f'{drive_path}data/complete/동코드(주소).csv')
dong_convert.head()

,구,법정동,행정동,code10,code8,code7
0,종로구,청운효자동,청운동,1111051500,1105150,110515
1,종로구,청운효자동,신교동,1111051500,1105150,110515
2,종로구,청운효자동,궁정동,1111051500,1105150,110515
3,종로구,청운효자동,효자동,1111051500,1105150,110515
4,종로구,청운효자동,창성동,1111051500,1105150,110515


In [ ]:
# 동 명칭에 . 포함
dong_convert['행정동'] = dong_convert['행정동'].apply(lambda x: re.sub(r'\.', '', x))
dong_convert['행정동'] = dong_convert['행정동'].apply(lambda x: re.sub(r'\·', '', x))

dong_convert['법정동'] = dong_convert['법정동'].apply(lambda x: re.sub(r'\.', '', x))
dong_convert['법정동'] = dong_convert['법정동'].apply(lambda x: re.sub(r'\·', '', x))

In [ ]:
df_merged = dong_convert.merge(df,
                               how = 'left',
                               right_on = '읍면동',
                               left_on = '행정동')

df_merged.head()

,구,법정동,행정동,code10,code8,code7,번호,개방서비스명,인허가일자,인허가취소일자,...,도로명전체주소,사업장명,업태구분명,좌표정보(x),좌표정보(y),업종,시군구,읍면동,lat,lon
0,종로구,청운효자동,청운동,1111051500,1105150,110515,2175,일반음식점,2023-07-26,NaN,...,"서울특별시 종로구 자하문로 131-37, 지하1층 (청운동)",타쳇(Tacet),경양식,197079.150243,NaN,일반음식점 경양식,종로구,청운동,NaN,NaN
1,종로구,청운효자동,청운동,1111051500,1105150,110515,5274,일반음식점,1992-11-18,NaN,...,서울특별시 종로구 자하문로 119 (청운동),양반촌,한식,197244.689084,NaN,일반음식점 한식,종로구,청운동,NaN,NaN
2,종로구,청운효자동,청운동,1111051500,1105150,110515,12925,일반음식점,1998-10-27,NaN,...,서울특별시 종로구 자하문로 118 (청운동),들풀식당,한식,197291.341182,NaN,일반음식점 한식,종로구,청운동,NaN,NaN
3,종로구,청운효자동,청운동,1111051500,1105150,110515,15543,일반음식점,2001-04-13,NaN,...,서울특별시 종로구 자하문로33길 2 (청운동),중국,중국식,197254.922522,NaN,일반음식점 중국식,종로구,청운동,NaN,NaN
4,종로구,청운효자동,청운동,1111051500,1105150,110515,21887,일반음식점,2016-03-18,NaN,...,"서울특별시 종로구 자하문로 112, 지하1층 (청운동)",서촌명가,한식,197299.640482,NaN,일반음식점 한식,종로구,청운동,NaN,NaN


In [ ]:
df_merged['구동'] = df_merged['시군구'] + ' ' + df_merged['법정동']
df_merged.head()

,구,법정동,행정동,code10,code8,code7,번호,개방서비스명,인허가일자,인허가취소일자,...,사업장명,업태구분명,좌표정보(x),좌표정보(y),업종,시군구,읍면동,lat,lon,구동
0,종로구,청운효자동,청운동,1111051500,1105150,110515,2175,일반음식점,2023-07-26,NaN,...,타쳇(Tacet),경양식,197079.150243,NaN,일반음식점 경양식,종로구,청운동,NaN,NaN,종로구 청운효자동
1,종로구,청운효자동,청운동,1111051500,1105150,110515,5274,일반음식점,1992-11-18,NaN,...,양반촌,한식,197244.689084,NaN,일반음식점 한식,종로구,청운동,NaN,NaN,종로구 청운효자동
2,종로구,청운효자동,청운동,1111051500,1105150,110515,12925,일반음식점,1998-10-27,NaN,...,들풀식당,한식,197291.341182,NaN,일반음식점 한식,종로구,청운동,NaN,NaN,종로구 청운효자동
3,종로구,청운효자동,청운동,1111051500,1105150,110515,15543,일반음식점,2001-04-13,NaN,...,중국,중국식,197254.922522,NaN,일반음식점 중국식,종로구,청운동,NaN,NaN,종로구 청운효자동
4,종로구,청운효자동,청운동,1111051500,1105150,110515,21887,일반음식점,2016-03-18,NaN,...,서촌명가,한식,197299.640482,NaN,일반음식점 한식,종로구,청운동,NaN,NaN,종로구 청운효자동


In [ ]:
df_merged['법정동'].isna().sum()

0

In [ ]:
# 소분류 기준으로 정리
df_store = df_merged.groupby(['구동','업종'])[['번호']].count().reset_index()
df_store = df_store.pivot(columns="업종",index = '구동', values = '번호')
df_store.reset_index(inplace = True)
df_store.fillna(0, inplace = True)
df_store.head()

업종,구동,동물미용업 동물미용업,동물병원 동물병원,동물약국 동물약국,미용업 기타,미용업 네일아트업,미용업 메이크업업,미용업 숙박업 기타,미용업 일반미용업,미용업 일반이용업,...,휴게음식점 유원지,휴게음식점 일반조리판매,휴게음식점 전통찻집,휴게음식점 철도역구내,휴게음식점 커피숍,휴게음식점 키즈카페,휴게음식점 패스트푸드,휴게음식점 편의점,휴게음식점 푸드트럭,휴게음식점 호프/통닭
0,강남구 개포1동,12.0,10.0,11.0,4.0,11.0,1.0,0.0,82.0,0.0,...,0.0,21.0,0.0,0.0,57.0,0.0,17.0,29.0,0.0,0.0
1,강남구 개포2동,16.0,14.0,21.0,5.0,21.0,4.0,0.0,129.0,0.0,...,0.0,32.0,0.0,0.0,94.0,0.0,27.0,45.0,0.0,0.0
2,강남구 개포3동,25.0,20.0,45.0,12.0,65.0,10.0,0.0,304.0,0.0,...,0.0,94.0,2.0,0.0,301.0,0.0,64.0,127.0,5.0,0.0
3,강남구 개포4동,12.0,10.0,11.0,4.0,11.0,1.0,0.0,82.0,0.0,...,0.0,21.0,0.0,0.0,57.0,0.0,17.0,29.0,0.0,0.0
4,강남구 논현1동,39.0,12.0,35.0,3.0,74.0,44.0,0.0,218.0,0.0,...,0.0,89.0,1.0,2.0,218.0,0.0,27.0,100.0,0.0,0.0


In [ ]:
# 중분류 기준으로 정리
df_store2 = df_merged.groupby(['구동','개방서비스명'])[['번호']].count().reset_index()
df_store2 = df_store2.pivot(columns='개방서비스명',index = '구동', values = '번호')
df_store2.reset_index(inplace = True)
df_store2.fillna(0, inplace = True)
df_store2.head()

개방서비스명,구동,동물미용업,동물병원,동물약국,미용업,병원,세탁업,약국,영화상영관,의원,이용업,일반음식점,제과점영업,휴게음식점
0,강남구 개포1동,12.0,10.0,11.0,114.0,1.0,28.0,34.0,0.0,103.0,14.0,401.0,19.0,164.0
1,강남구 개포2동,16.0,14.0,21.0,181.0,2.0,41.0,60.0,0.0,148.0,19.0,624.0,33.0,254.0
2,강남구 개포3동,25.0,20.0,45.0,480.0,6.0,68.0,110.0,0.0,368.0,34.0,2032.0,74.0,789.0
3,강남구 개포4동,12.0,10.0,11.0,114.0,1.0,28.0,34.0,0.0,103.0,14.0,401.0,19.0,164.0
4,강남구 논현1동,39.0,12.0,35.0,478.0,16.0,52.0,77.0,19.0,412.0,12.0,2055.0,49.0,534.0


In [ ]:
store = df_store.merge(df_store2,
               how = 'left',
               on = '구동')
store.head()

,구동,동물미용업 동물미용업,동물병원 동물병원,동물약국 동물약국,미용업 기타,미용업 네일아트업,미용업 메이크업업,미용업 숙박업 기타,미용업 일반미용업,미용업 일반이용업,...,미용업,병원,세탁업,약국,영화상영관,의원,이용업,일반음식점,제과점영업,휴게음식점
0,강남구 개포1동,12.0,10.0,11.0,4.0,11.0,1.0,0.0,82.0,0.0,...,114.0,1.0,28.0,34.0,0.0,103.0,14.0,401.0,19.0,164.0
1,강남구 개포2동,16.0,14.0,21.0,5.0,21.0,4.0,0.0,129.0,0.0,...,181.0,2.0,41.0,60.0,0.0,148.0,19.0,624.0,33.0,254.0
2,강남구 개포3동,25.0,20.0,45.0,12.0,65.0,10.0,0.0,304.0,0.0,...,480.0,6.0,68.0,110.0,0.0,368.0,34.0,2032.0,74.0,789.0
3,강남구 개포4동,12.0,10.0,11.0,4.0,11.0,1.0,0.0,82.0,0.0,...,114.0,1.0,28.0,34.0,0.0,103.0,14.0,401.0,19.0,164.0
4,강남구 논현1동,39.0,12.0,35.0,3.0,74.0,44.0,0.0,218.0,0.0,...,478.0,16.0,52.0,77.0,19.0,412.0,12.0,2055.0,49.0,534.0


In [ ]:
store.to_csv(f'{drive_path}final/data/store.csv', index = False)

## 지도 그리기

In [ ]:
def merge_map(df_name, key):

    region_code = pd.read_csv(f'{drive_path}final/data/region_code.csv')
    dong_shp = gpd.read_file(f'{drive_path}data/seoul_shp/dong/bnd_dong_11_2023_2023_2Q.shp')

    # 코드명 타입 일치
    region_code['ADM_CD'] = region_code['ADM_CD'].astype('int')
    dong_shp['ADM_CD'] = dong_shp['ADM_CD'].astype('int')

    # 지도파일과 지역코드 결합
    map_base = dong_shp.merge(region_code,
                          how = 'left',
                          on = 'ADM_CD')
    map_base['ADM_NM'] = map_base['ADM_NM'].apply(lambda x: re.sub(r'\·', '', x))

    # 동이름 다른것에 대응하기 위해서
    map_base['key1'] = map_base['시군구'] + ' ' + map_base['ADM_NM']
    map_base['key2'] = map_base['시군구'] + ' ' + map_base['읍면동']

    if '구동' in df_name.columns:
        pass
    else :
        df_name['구동'] = df_name['자치구'] + ' ' + df_name['행정동명']

    globals()['final'] = map_base.merge(df_name.rename(columns = {'구동' : key}),
                                        how = 'left',
                                        on = key)

    final.dropna(inplace = True)

In [ ]:
merge_map(store, 'key2')
selected_columns = ['ADM_CD'] + final.columns[10:].tolist()
final = final[selected_columns]
final
final.head()

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,...,미용업,병원,세탁업,약국,영화상영관,의원,이용업,일반음식점,제과점영업,휴게음식점
0,20230701,사직동,11010530,"POLYGON ((953553.932 1953335.741, 953555.211 1...",서울특별시 종로구 사직동,1101053.0,1.111053e+09,11010530.0,11110530.0,11110530.0,...,74.0,2.0,8.0,14.0,4.0,55.0,10.0,1094.0,18.0,224.0
1,20230701,삼청동,11010540,"POLYGON ((954025.242 1953916.389, 954026.972 1...",서울특별시 종로구 삼청동,1101054.0,1.111054e+09,11010540.0,11110540.0,11110540.0,...,4.0,0.0,1.0,3.0,0.0,7.0,0.0,272.0,3.0,91.0
2,20230701,부암동,11010550,"POLYGON ((952490.380 1956548.821, 952497.594 1...",서울특별시 종로구 부암동,1101055.0,1.111055e+09,11010550.0,11110550.0,11110550.0,...,12.0,0.0,5.0,1.0,0.0,4.0,1.0,141.0,7.0,65.0
3,20230701,평창동,11010560,"POLYGON ((953683.828 1959209.871, 953665.283 1...",서울특별시 종로구 평창동,1101056.0,1.111056e+09,11010560.0,11110560.0,11110560.0,...,24.0,1.0,8.0,5.0,0.0,9.0,2.0,132.0,4.0,75.0
4,20230701,한남동,11030740,"POLYGON ((956238.296 1950166.610, 956237.942 1...",서울특별시 용산구 한남동,1103074.0,1.117068e+09,11030740.0,11170685.0,11170685.0,...,95.0,0.0,8.0,6.0,0.0,24.0,8.0,778.0,29.0,120.0


In [ ]:
final.columns

Index(['BASE_DATE', 'ADM_NM', 'ADM_CD', 'geometry', 'dong_name', '통계청코드',
       '도로명코드10', '통계청코드8', '도로명코드8', '도로명코드',
       ...
       '미용업', '병원', '세탁업', '약국', '영화상영관', '의원', '이용업', '일반음식점', '제과점영업',
       '휴게음식점'],
      dtype='object', length=110)

In [ ]:
final[final['미용업 네일아트업'].isna()]

,BASE_DATE,ADM_NM,ADM_CD,geometry,dong_name,통계청코드,도로명코드10,통계청코드8,도로명코드8,도로명코드,...,미용업,병원,세탁업,약국,영화상영관,의원,이용업,일반음식점,제과점영업,휴게음식점


In [ ]:
final.to_csv(f'{drive_path}final/data/song2.csv', index = False)